# Телеграм приложение, качает сообщения из телеграм чатов

In [1]:
# https://proglib.io/p/pishem-prostoy-grabber-dlya-telegram-chatov-na-python-2019-11-06

In [2]:
import requests
from bs4 import BeautifulSoup, SoupStrainer
import numpy as np

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
url = 'https://coingape.com/best-telegram-crypto-channels-list/'

r = requests.get(url, headers=headers)


links = []
i = 0
for link in BeautifulSoup(r.content, parse_only=SoupStrainer('a')):
    if i == 0:
        i += 1
        continue
    if link.has_attr('href'):
        if link['href'].lower().startswith('https://t.me/'):
            links.append(link['href'])
links = np.unique(links)

links[:5]

array(['HTTPS://t.me/Steve_Admin', 'https://T.Me/Bitcoin',
       'https://T.Me/Bitcoinistnews', 'https://T.Me/Coinfarm',
       'https://T.Me/Cointelegraph'], dtype='<U44')

In [3]:
len(links)

69

In [25]:
import configparser
import json

from telethon.sync import TelegramClient
from telethon import connection
from telethon import errors


# для корректного переноса времени сообщений в json
from datetime import date, datetime

# классы для работы с каналами
from telethon.tl.functions.channels import GetParticipantsRequest
from telethon.tl.types import ChannelParticipantsSearch

# класс для работы с сообщениями
from telethon.tl.functions.messages import GetHistoryRequest

import pandas as pd
pd.set_option("display.max_columns",  200)



# Считываем учетные данные
config = configparser.ConfigParser()
config.read("config.ini")

# Присваиваем значения внутренним переменным
api_id   = config['Telegram']['api_id']
api_hash = config['Telegram']['api_hash']
username = config['Telegram']['username']


# async def dump_all_participants(channel):
#     """Записывает json-файл с информацией о всех участниках канала/чата"""
#     offset_user = 0    # номер участника, с которого начинается считывание
#     limit_user = 100   # максимальное число записей, передаваемых за один раз

#     all_participants = []   # список всех участников канала
#     filter_user = ChannelParticipantsSearch('')

#     while True:
#         participants = await client(GetParticipantsRequest(
#             channel, filter_user, offset_user, limit_user, hash=0))
#         if not participants.users:
#             break
#         all_participants.extend(participants.users)
#         offset_user += len(participants.users)
#     return all_participants
        
        
async def dump_all_messages(channel, num):
    """Записывает json-файл с информацией о всех сообщениях канала/чата"""
    offset_msg = 0    # номер записи, с которой начинается считывание
    limit_msg = 100   # максимальное число записей, передаваемых за один раз

    all_messages = []   # список всех сообщений
    total_messages = 0
    total_count_limit = 0  # поменяйте это значение, если вам нужны не все сообщения

#     class DateTimeEncoder(json.JSONEncoder):
#         '''Класс для сериализации записи дат в JSON'''
#         def default(self, o):
#             if isinstance(o, datetime):
#                 return o.isoformat()
#             if isinstance(o, bytes):
#                 return list(o)
#             return json.JSONEncoder.default(self, o)

    while True:
        history = await client(GetHistoryRequest(
            peer=channel,
            offset_id=offset_msg,
            offset_date=None, add_offset=0,
            limit=limit_msg, max_id=0, min_id=0,
            hash=0))
        if not history.messages:
            break
        messages = history.messages
        for message in messages:
            all_messages.append(message.to_dict())
        offset_msg = messages[len(messages) - 1].id
        total_messages = len(all_messages)
        if total_count_limit != 0 and total_messages >= total_count_limit:
            break

#     with open('channel_messages.json', 'w', encoding='utf8') as outfile:
#         json.dump(all_messages, outfile, ensure_ascii=False, cls=DateTimeEncoder)
    pd.DataFrame(all_messages).to_csv(f'channel_{num}.csv', index=False)     
            
client = TelegramClient(username, api_id, api_hash)


# proxy = (proxy_server, proxy_port, proxy_key)

# client = TelegramClient(username, api_id, api_hash,
#     connection=connection.ConnectionTcpMTProxyRandomizedIntermediate,
#     proxy=proxy)


client.start()
import asyncio
import nest_asyncio
nest_asyncio.apply()
from tqdm import tqdm

client.connect()

all_messages = 0

async def main():
    for num, url in tqdm(enumerate(links)):
        print(num, url)
        try:
            channel = await client.get_entity(links[18])
        except ValueError:
            pass
#         await dump_all_participants(channel)
        await dump_all_messages(channel, num)
        
    
async with client:
    client.loop.run_until_complete(main())
#     client.disconnect()

#     client.run_until_disconnected()

# client.disconnect()

/var/folders/jx/_kswgcgx081f3lcl2srys7rh0000gn/T/ipykernel_37832/1449463897.py:101: RuntimeWarning: coroutine 'AuthMethods._start' was never awaited
  client.start()
/var/folders/jx/_kswgcgx081f3lcl2srys7rh0000gn/T/ipykernel_37832/1449463897.py:107: RuntimeWarning: coroutine 'TelegramBaseClient.connect' was never awaited
  client.connect()
0it [00:00, ?it/s]

0 HTTPS://t.me/Steve_Admin


1it [00:05,  5.90s/it]

1 https://T.Me/Bitcoin


2it [00:10,  4.98s/it]

2 https://T.Me/Bitcoinistnews


3it [00:13,  4.30s/it]

3 https://T.Me/Coinfarm


4it [00:35, 11.11s/it]

# ячейку выше всю не запускал, долго

In [5]:
dfs = []
import pandas as pd
for i in range(len(links)):
    try:
        dfs.append(pd.read_csv(f'channel_{i}.csv'))
    except FileNotFoundError:
        pass

In [6]:
data = pd.concat(dfs)

In [7]:
data.to_csv('all_chats.csv', index=False)

In [10]:
# len(pd.read_csv('Bitcoin_tweets.csv'))
# 1200k там

In [12]:
len(data)

692246

In [13]:
data.sample(5)

,_,id,peer_id,date,message,out,mentioned,media_unread,silent,post,...,entities,views,forwards,replies,edit_date,post_author,grouped_id,restriction_reason,ttl_period,action
11824,Message,14354,"{'_': 'PeerChannel', 'channel_id': 1230368984}",2019-08-22 09:33:06+00:00,📢 Upbit announced a new coin: TSHP,False,False,False,False,True,...,[],5181.0,0.0,NaN,NaN,NaN,NaN,[],NaN,NaN
8617,Message,17990,"{'_': 'PeerChannel', 'channel_id': 1230368984}",2020-04-30 10:06:17+00:00,✅✅✅✅\n\njoin Vip of horror\nhttps://forms.gle/...,False,False,False,False,True,...,"[{'_': 'MessageEntityUrl', 'offset': 25, 'leng...",8323.0,2.0,NaN,NaN,NaN,NaN,[],NaN,NaN
12325,Message,13772,"{'_': 'PeerChannel', 'channel_id': 1230368984}",2019-06-26 21:37:53+00:00,i think we all should remember this day 26.6.2...,False,False,False,True,True,...,[],10435.0,59.0,NaN,NaN,NaN,NaN,[],NaN,NaN
7873,MessageService,18865,"{'_': 'PeerChannel', 'channel_id': 1230368984}",2020-06-07 15:27:35+00:00,NaN,False,False,False,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'_': 'MessageActionPinMessage'}
8298,Message,18379,"{'_': 'PeerChannel', 'channel_id': 1230368984}",2020-05-21 14:15:11+00:00,✅,False,False,False,False,True,...,[],5593.0,1.0,NaN,NaN,NaN,NaN,[],NaN,NaN


# В итоге было решено не использовать эти данные из-за их неоднородности и того что все модели которые можно позаимствовать написаны с твитера

# Но можно выкачать отдельные каналы похожие на твиттер